In [52]:
import numpy as np
import pandas as pd
import os
import subprocess

# get the path to the root of the repository
root_path = (
    subprocess.check_output(["git", "rev-parse", "--show-toplevel"])
    .decode("utf-8")
    .strip()
)
# set the working directory to the root of the repository
os.chdir(root_path)
# create ./data/09.format_result/ if it doesn't exist
os.makedirs("data/09.format_result", exist_ok=True)

In [60]:
df = pd.read_csv("./data/07.DEG/star2counts.tsv", delimiter="\t")
# discard GeneType
# df = df.drop(columns=['GeneType'])
# add a column named "LSK-WT6 Mean", calculate the mean of LSK-WT6_*
df["LSK-WT6 Mean"] = df.loc[:, df.columns.str.startswith("LSK-WT6")].mean(axis=1)
# add a column named "LSK-KO10 Mean", calculate the mean of LSK-KO10_*
df["LSK-KO10 Mean"] = df.loc[:, df.columns.str.startswith("LSK-KO10")].mean(axis=1)
# add a column named "LSK-KO12 Mean", calculate the mean of LSK-KO12_*
df["LSK-KO12 Mean"] = df.loc[:, df.columns.str.startswith("LSK-KO12")].mean(axis=1)
deg_AB = pd.read_csv("data/07.DEG/DEG_DESeq2_BvsA.tsv", sep="\t")
deg_AB = deg_AB[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AB.columns = [str(col) + "_WT6/KO10" for col in deg_AB.columns]
deg_AC = pd.read_csv("data/07.DEG/DEG_DESeq2_CvsA.tsv", sep="\t")
deg_AC = deg_AC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AC.columns = [str(col) + "_WT6/KO12" for col in deg_AC.columns]
deg_BC = pd.read_csv("data/07.DEG/DEG_DESeq2_CvsB.tsv", sep="\t")
deg_BC = deg_BC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_BC.columns = [str(col) + "_KO10/KO12" for col in deg_BC.columns]
# join df and deg_AB by GeneID and GeneID_WT/KO4
df = df.merge(deg_AB, left_on="GeneID", right_on="GeneID_WT6/KO10", how="left")
df = df.merge(deg_AC, left_on="GeneID", right_on="GeneID_WT6/KO12", how="left")
df = df.merge(deg_BC, left_on="GeneID", right_on="GeneID_KO10/KO12", how="left")
# drop GeneID.*, but keep GeneID
df = df.drop(columns=df.columns[df.columns.str.startswith("GeneID_")])
print(df.head())

                  GeneID            GeneName              GeneType  LSK-WT6_1  \
0   ENSMUSG00000103922.2              Gm6123  processed_pseudogene          4   
1  ENSMUSG00000033845.14              Mrpl15        protein_coding       2348   
2   ENSMUSG00000102275.2             Gm37144                   TEC         19   
3   ENSMUSG00000120403.1  ENSMUSG00000120403                lncRNA         10   
4  ENSMUSG00000025903.15              Lypla1        protein_coding       1977   

   LSK-WT6_2  LSK-KO10_1  LSK-KO10_2  LSK-KO12_1  LSK-KO12_2  LSK-WT6 Mean  \
0          5          10           8           6          11           4.5   
1       2032        2242        2508        2836        3048        2190.0   
2         22          66          41          25          24          20.5   
3          4          12           4           9           1           7.0   
4       1641        1998        2222        1498        1573        1809.0   

   ...  LSK-KO12 Mean  log2FoldChange_WT6/KO

In [55]:
import importlib
if importlib.util.find_spec("Bio") is None:
    !pip install Bio

from Bio import Entrez
from urllib.error import HTTPError
import pickle

os.environ["http_proxy"] = "http://192.168.206.22:7890"


def get_gene_description(ensembl_id):
    # set the email address for Entrez
    Entrez.email = "babaolanqiu@gmail.com"

    # remove the transcript version number from the ensembl ID
    ensembl_id = ensembl_id.split(".")[0]
    # use Entrez to search for the gene information
    handle = Entrez.esearch(db="gene", term=ensembl_id)
    record = Entrez.read(handle)

    try:
        # extract the gene ID from the search results
        gene_id = record["IdList"][0]

        # use Entrez to retrieve the gene information
        handle = Entrez.efetch(db="gene", id=gene_id, retmode="xml")
        record = Entrez.read(handle)

        # extract the gene full name from the gene information
        gene_description = record[0]["Entrezgene_gene"]["Gene-ref"]["Gene-ref_desc"]
        # return pd dataframe of GeneID and Gene description
        return pd.DataFrame(
            {"GeneID": ensembl_id, "Gene description": gene_description}, index=[0]
        )
    except IndexError:
        print(f"No gene information found for Ensembl ID {ensembl_id}")
        return pd.DataFrame(
            {"GeneID": ensembl_id, "Gene description": "None"}, index=[0]
        )
    except HTTPError:
        print(f"HTTPError for Ensembl ID {ensembl_id}")
        # re-try thistry
        return get_gene_description(ensembl_id)
    except KeyError:
        gene_description = record[0]["Entrezgene_gene"]["Gene-ref"]["Gene-ref_formal-name"]["Gene-nomenclature"]["Gene-nomenclature_name"]
        return pd.DataFrame(
            {"GeneID": ensembl_id, "Gene description": gene_description}, index=[0]
        )


# apply the function to every 5 items in the "GeneID" column of df, use for loop
# gene_descriptions = pd.DataFrame(columns=["GeneID", "Gene description"],index=range(0))
# if gene_descriptions variable don't exist, create it
try:
    gene_descriptions
except NameError:
    gene_descriptions = pd.DataFrame(
        columns=["GeneID", "Gene description"], index=range(0)
    )
for ensembl_id in df["GeneID"]:
    # if GeneID exist in gene_descriptions, skip
    ensembl_id_2 = ensembl_id.split(".")[0]
    if ensembl_id_2 in gene_descriptions["GeneID"].values:
        continue
    # if GeneID not exist in gene_descriptions, apply get_gene_description, and append to the end of gene_descriptions
    gene_descriptions = pd.concat(
        [gene_descriptions, get_gene_description(ensembl_id)], ignore_index=True
    )
    # print progress
    print(f"Progress: {gene_descriptions.shape[0]}/{df.shape[0]}")
    # make a checkpoint every 100 times
    i = gene_descriptions.shape[0]
    # if i is times of 100
    if i % 100 == 0:
        with open("data/09.format_result.pickle", "wb") as f:
            pickle.dump([df, gene_descriptions], f)
        print(f"Progress saved to data/09.format_result.pickle")
# merge df and gene_descriptions by GeneID

Progress: 18501/18549
Progress: 18502/18549
Progress: 18503/18549
No gene information found for Ensembl ID ENSMUSG00000120664
Progress: 18504/18549
Progress: 18505/18549
Progress: 18506/18549
Progress: 18507/18549
Progress: 18508/18549
Progress: 18509/18549
Progress: 18510/18549
Progress: 18511/18549
Progress: 18512/18549
No gene information found for Ensembl ID ENSMUSG00000087159
Progress: 18513/18549
No gene information found for Ensembl ID ENSMUSG00000081137
Progress: 18514/18549
Progress: 18515/18549
Progress: 18516/18549
No gene information found for Ensembl ID ENSMUSG00000087263
Progress: 18517/18549
No gene information found for Ensembl ID ENSMUSG00000086695
Progress: 18518/18549
No gene information found for Ensembl ID ENSMUSG00000095562
Progress: 18519/18549
Progress: 18520/18549
Progress: 18521/18549
No gene information found for Ensembl ID ENSMUSG00000099876
Progress: 18522/18549
Progress: 18523/18549
Progress: 18524/18549
No gene information found for Ensembl ID ENSMUSG0000

In [61]:
df["GeneID_2"] = df["GeneID"].str.split(".").str[0]
df = pd.merge(df, gene_descriptions, left_on="GeneID_2", right_on="GeneID")
# remove GeneID_2
df = df.drop(columns=["GeneID_2", "GeneID_y"])
# move gene description to the third column
cols = list(df.columns)
cols = cols[:2] + [cols[-1]] + cols[2:-1]
df = df[cols]
# rename GeneID_x to GeneID
df = df.rename(columns={"GeneID_x": "GeneID"})
df.to_csv("data/09.format_result/09.format_result.tsv", sep="\t", index=False)

In [57]:
df_cpm = pd.read_csv("./data/07.DEG/star2CPM.tsv", sep="\t")
# discard GeneType
# df_cpm = df_cpm.drop(columns=['GeneType'])
# add a column named "LSK-WT6 Mean", calculate the mean of LSK-WT6_*
df_cpm["LSK-WT6 Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("LSK-WT6")].mean(
    axis=1
)
# add a column named "LSK-KO10 Mean", calculate the mean of LSK-KO10_*
df_cpm["LSK-KO10 Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("LSK-KO10")].mean(
    axis=1
)
# add a column named "LSK-KO12 Mean", calculate the mean of LSK-KO12_*
df_cpm["LSK-KO12 Mean"] = df_cpm.loc[:, df_cpm.columns.str.startswith("LSK-KO12")].mean(
    axis=1
)
deg_AB = pd.read_csv("data/07.DEG/DEG_DESeq2_BvsA.tsv", sep="\t")
deg_AB = deg_AB[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AB.columns = [str(col) + "_WT6/KO10" for col in deg_AB.columns]
deg_AC = pd.read_csv("data/07.DEG/DEG_DESeq2_CvsA.tsv", sep="\t")
deg_AC = deg_AC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_AC.columns = [str(col) + "_WT6/KO12" for col in deg_AC.columns]
deg_BC = pd.read_csv("data/07.DEG/DEG_DESeq2_CvsB.tsv", sep="\t")
deg_BC = deg_BC[["GeneID", "log2FoldChange", "pvalue", "padj"]]
deg_BC.columns = [str(col) + "_KO10/KO12" for col in deg_BC.columns]
# join df_cpm and deg_AB by GeneID and GeneID_WT6/KO10
df_cpm = df_cpm.merge(deg_AB, left_on="GeneID", right_on="GeneID_WT6/KO10", how="left")
df_cpm = df_cpm.merge(deg_AC, left_on="GeneID", right_on="GeneID_WT6/KO12", how="left")
df_cpm = df_cpm.merge(deg_BC, left_on="GeneID", right_on="GeneID_KO10/KO12", how="left")
# drop GeneID.*, but keep GeneID
df_cpm = df_cpm.drop(columns=df_cpm.columns[df_cpm.columns.str.startswith("GeneID_")])
print(df_cpm.head())

df_cpm["GeneID_2"] = df_cpm["GeneID"].str.split(".").str[0]
df_cpm = pd.merge(df_cpm, gene_descriptions, left_on="GeneID_2", right_on="GeneID")
# remove GeneID_2
df_cpm = df_cpm.drop(columns=["GeneID_2", "GeneID_y"])
# move gene description to the third column
cols = list(df_cpm.columns)
cols = cols[:2] + [cols[-1]] + cols[2:-1]
df_cpm = df_cpm[cols]
# rename GeneID_x to GeneID
df_cpm = df_cpm.rename(columns={"GeneID_x": "GeneID"})
df_cpm.to_csv("data/09.format_result/09.format_result_CPM.tsv", sep="\t", index=False)

                  GeneID            GeneName              GeneType  LSK-WT6_1  \
0   ENSMUSG00000103922.2              Gm6123  processed_pseudogene   0.101076   
1  ENSMUSG00000033845.14              Mrpl15        protein_coding  59.331856   
2   ENSMUSG00000102275.2             Gm37144                   TEC   0.480113   
3   ENSMUSG00000120403.1  ENSMUSG00000120403                lncRNA   0.252691   
4  ENSMUSG00000025903.15              Lypla1        protein_coding  49.957019   

   LSK-WT6_2  LSK-KO10_1  LSK-KO10_2  LSK-KO12_1  LSK-KO12_2  LSK-WT6 Mean  \
0   0.138871    0.281539    0.201564    0.166231    0.287126      0.119974   
1  56.437271   63.121100   63.190225   78.571693   79.559873     57.884563   
2   0.611033    1.858159    1.033014    0.692628    0.626456      0.545573   
3   0.111097    0.337847    0.100782    0.249346    0.026102      0.181894   
4  45.577540   56.251542   55.984322   41.502255   41.058950     47.767279   

   ...  LSK-KO12 Mean  log2FoldChange_WT6/KO

The following code is used to load or save gene_description data

In [58]:
import pickle

# save all the dataframes to a pickle file
with open("data/09.format_result/09.format_result.pickle", "wb") as f:
    pickle.dump([df, gene_descriptions], f)

In [54]:
import pickle

# load all the dataframes from a pickle file
with open("data/09.format_result.pickle", "rb") as f:
    df, gene_descriptions = pickle.load(f)